# Задача

В мобильной игре, после установки и запуска игры начинается туториал - игрок проходит обучение игре, которое состоит из нескольких последовательных шагов. Данные представляют собой набор строк - уникальный id игрока, datetime и номер шага туториала который пройден игроком.

Геймдизайнеры выдвинули гипотезу - если мы с высокой вероятностью сможем предсказать шаг на котором игрок уйдет из игры заранее, то запустив альтернативное продолжение туториала с этого шага мы избежим оттока и увеличим прохождение туториала.

- Как бы вы решали задачу предсказания отвала игрока?
- Как оценить качество полученной модели? Опишите методы которые знаете и какой считаете оптимальным и почему.
- Предположим что вы разработали модель которая с достаточной вероятностью предсказывает отвал. Опишите дизайн эксперимента и методологию по проверке гипотезы геймдизайнеров

# Решение

Перед тем, как принимать решение о выборе модели, стоит провести EDA. Возможно, удастся выявить какие-нибудь закономерности и инсайты того, где и почему отваливаются игроки. Если максимальный отток происходит на каком-то конкретном шаге, то стоит посмотреть, что именно там нужно сделать или что показывается. Например, если мы увидим, что люди массово закрывают туториал, где мы предлагаем привязать карту или совершить «очень выгодную» внутриигровую покупку, то имеет смысл задвинуть это на более поздние стадии игры. 

Если ничего интересного найти не удалось, то можно попробовать обучить модель, которая будет нам предсказывать, отвалится ли игрок или нет. Для этого нам, конечно же, понадобятся данные, но лишь указанных данных нам может не хватить для высокого качества, так как по факту в наличии только datetime и номер шага. Посмотрим на синт. примере.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# синтетические даты
datetime = pd.date_range(start='2023-12-01', periods=10, freq='1D').tolist()

# датафрейм
df = pd.DataFrame(data=datetime, columns=['datetime'])

# синтетические номера шагов
df['step'] = [9, 1, 1, 2, 1, 6, 8, 1, 3, 3]

In [3]:
df

,datetime,step
0,2023-12-01,9
1,2023-12-02,1
2,2023-12-03,1
3,2023-12-04,2
4,2023-12-05,1
5,2023-12-06,6
6,2023-12-07,8
7,2023-12-08,1
8,2023-12-09,3
9,2023-12-10,3


In [4]:
df['step'].value_counts()

1    4
3    2
9    1
2    1
6    1
8    1
Name: step, dtype: int64

Видим, что из датафрейма с 10 игроками 4 остановились на первом этапе. Это может говорить о возможной закономерности, но не доказывает её. Если мы захотим обучить модель машинного обучения для предсказания шага для оттока, то нам потребуется больше фич. Во-первых, потребуется разметка тех, кто отвалился, а кто — нет. Благо, эти данные у нас в целом должны присутствовать. 

In [5]:
df['is_gone'] = [0, 1, 1, 1, 1, 0, 0, 1, 1, 0]
df

,datetime,step,is_gone
0,2023-12-01,9,0
1,2023-12-02,1,1
2,2023-12-03,1,1
3,2023-12-04,2,1
4,2023-12-05,1,1
5,2023-12-06,6,0
6,2023-12-07,8,0
7,2023-12-08,1,1
8,2023-12-09,3,1
9,2023-12-10,3,0


Во-вторых, этих данных тоже вряд ли хватит, так как шагов у туториала вряд ли будет очень много. Из столбца datetime мы тоже сможем получить полезную информацию, например, о том, сколько всего времени каждый игрок проходил все шаги. А если есть подозрение на влияние времени суток или дня недели, то это тоже можно будет вытащить. 

In [6]:
# как, например, может выглядеть продолжительность (сейчас неважно, что за единицы измерения)
df['duration'] = [19, 3, 2, 4, 3, 14, 16, 6, 5, 6]
df

,datetime,step,is_gone,duration
0,2023-12-01,9,0,19
1,2023-12-02,1,1,3
2,2023-12-03,1,1,2
3,2023-12-04,2,1,4
4,2023-12-05,1,1,3
5,2023-12-06,6,0,14
6,2023-12-07,8,0,16
7,2023-12-08,1,1,6
8,2023-12-09,3,1,5
9,2023-12-10,3,0,6


Сюда можно будет добавить дополнительные признаки по каждому клиенту, например, платформа, откуда пришел и тп. Дальнейшая логика понятна.

Теперь, что касается обучения моделей. Каких именно? На самом деле трудно сказать сразу, потребуется поэкспериментировать. Сама задача у нас будет вначале бинарной классификации (отвалится или нет). Это важный момент, нам для начала нужно хотя бы предсказывать то, что игрок не дойдет до конца обучения. А уже потом можно будет масштабировать на запуск предсказания именно что номера шага, на котором это произойдет. То есть можно даже сделать ансамбль моделей (стекинг), где одна модель будет предсказывать отток, другая — пытаться предсказать шаг, на котором это произойдет.

| Детали     | Описание                                                                                                                                                                  |
|------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Модели     | **logistic regression**, **random forest** для классификации или **gradient boosting** для того же. Разница в предсказываемых классах для моделей будет отличаться (0 и 1 или N шагов туториала). Для логистической регрессии можно будет поиграться с отсечкой вероятностей.                         |
| Инференс   | **Скорость инференса** может быть разной, а тип предсказания явно будет **синхронный**, подстраивающийся под каждого игрока, который может начать туториал в любое время. |
| Размещение | **Размещение модели: embedded или облако**. В первом случае увеличится вес приложения, а во втором потребуется доступ в интернет.                                         |
| Метрики    | В качестве метрик должны подойти **F1-мера** и **ROC-AUC**, они отразят точность и полноту предсказаний.                                                                          |

Но это только полдела, дальше потребуется проверить гипотезу геймдизайнеров. В этом плане нас как обычно выручит сплит-тестирование, где нам нужно будет создать две или более группы игроков: одни будут играть как сейчас, а другие с нововведениями. Статистику мы скорее всего будем использовать накопительную, а модели дообучать даже после деплоя. Далее потребуется провести сам эксперимент, предсказывая на каждом шаге туториала вероятность отвала, либо же ожидая предсказанного шага, после чего пуская игроков по альтернативному пути и ожидая, что предсказание оттока не сбудется. Это сложный эксперимент, так как предполагает высокое качество моделей и встраивания их в архитектуру проекта.